In [2]:
import numpy as np
import pandas as pd
import keras

DISPLAY = False

In [3]:
from bc_helper.simulator_data import SimulatorData
from bc_helper.load import load_smooth_data, load_simple_data

orig = load_simple_data()
smooth = load_smooth_data()

orig_data = SimulatorData(orig)
smooth_data = SimulatorData(smooth)

len(df): 12557


In [3]:
def data_info(sim_data, name):
  # Number of training examples
  n_train = sim_data.num_train
  # Number of testing examples.
  n_test = sim_data.num_validation
  # What's the shape of an image?
  image_shape = sim_data.feature_shape
  # Max and min steering angles
  s_max = sim_data.max_label
  s_min = sim_data.min_label

  print(name, "number of training examples =", n_train)
  print(name, "number of testing examples =", n_test)
  print(name, "image data shape =", image_shape)
  print(name, "max steering angle =", s_max)
  print(name, "min steering angle =", s_min)
  
if DISPLAY:
  data_info(orig_data, "Steering Data")
  print()
  data_info(smooth_data, "Smoothed Steering Data")

In [4]:
import matplotlib.pyplot as plt
import random
# Visualizations will be shown in the notebook.
%matplotlib inline

def display_image(index, data):
  print("Steering angle:", data._df['steering'][index])
  fig = plt.figure(figsize = (3, 3))
  axis = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
  axis.imshow(data.img(index))
  plt.show()
  print("---------------------------------------------------------\n")
  
if DISPLAY:
  for i in range(10):
    display_image(random.randint(0, orig_data.num_train), orig_data)

In [5]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline

def hist(data, name):
  n, bins, patches = plt.hist(data, bins=50, facecolor='green')

  plt.xlabel('steering angle')
  plt.ylabel('Count')
  plt.title('Count  of ' + name)
  plt.grid(True)
  plt.show()
  
if DISPLAY:
  hist(orig_data._df['steering'], 'steering angle')
  hist([sa for sa in orig_data._df['steering'] if sa > 0.01 or sa < -0.01], 'steering angle without zeros')
  
  hist(smooth_data._df['smooth_steering'], 'smooth steering angle')
  hist([sa for sa in smooth_data._df['smooth_steering'] if sa > 0.01 or sa < -0.01], 'smooth steering angle without zeros')

Ok! So we clearly have a ton of zeros. but when the zeros are taken out the data is skewed left. This makes sense because both courses were basically counter clockwise circles. 

One fun note, a model could get a really high training by just guessing 0 the whole time. 

In [6]:
# from keras.applications.vgg16 import VGG16
from keras.layers import Dropout, Input
import h5py
from bc_helper.bottleneck_data import load_bottleneck_model

orig_precalc_train, orig_precalc_val = load_bottleneck_model('original')
orig_X_train = orig_precalc_train['features']
orig_y_train = orig_precalc_train['labels']
orig_X_val = orig_precalc_val['features']
orig_y_val = orig_precalc_val['labels']
print("orig train:", orig_X_train.shape, orig_y_train.shape)
print("orig_val:", orig_X_val.shape, orig_y_val.shape)

smooth_precalc_train, smooth_precalc_val = load_bottleneck_model('smooth')
smooth_X_train = smooth_precalc_train['features']
smooth_y_train = smooth_precalc_train['labels']
smooth_X_val = orig_precalc_val['features']
smooth_y_val = orig_precalc_val['labels']
print("smooth train", smooth_X_train.shape, smooth_y_train.shape)
print("smooth val:", smooth_X_val.shape, smooth_y_val.shape)

orig train: (11410, 5, 10, 512) (11410,)
orig_val: (4891, 5, 10, 512) (4891,)
smooth train (8789, 5, 10, 512) (8789,)
smooth val: (4891, 5, 10, 512) (4891,)


In [7]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.models import Sequential

def eval_model(X_train, y_train, X_val, y_val):

    # define model
    input_shape = X_train.shape[1:]
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    print("train model")
    model.fit(X_train, y_train, batch_size=3, nb_epoch=10, verbose=2, validation_data=(X_val, y_val), shuffle=True)

    # evaluate model
    print("Evaluate train")
    train_results = model.evaluate(X_train, y_train, batch_size=32, verbose=2)
    train_predictions = model.predict(X_train, batch_size=32, verbose=2)

    print("Evaluate validation")
    val_results = model.evaluate(X_val, y_val, batch_size=32, verbose=2)
    val_predictions = model.predict(X_val, batch_size=32, verbose=2)
    
    return [model, train_results, train_predictions, val_results, val_predictions]
  
orig_results = eval_model(orig_X_train, orig_y_train, orig_X_val, orig_y_val)
smooth_results = eval_model(smooth_X_train, smooth_y_train, smooth_X_val, smooth_y_val)





train model
Train on 11410 samples, validate on 4891 samples
Epoch 1/10
365s - loss: 415.5227 - acc: 0.2598 - val_loss: 0.0622 - val_acc: 0.6132
Epoch 2/10
361s - loss: 0.9315 - acc: 0.5647 - val_loss: 0.0612 - val_acc: 0.6140
Epoch 3/10
363s - loss: 0.1878 - acc: 0.6027 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 4/10
359s - loss: 0.0994 - acc: 0.6079 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 5/10
359s - loss: 0.0725 - acc: 0.6110 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 6/10
360s - loss: 0.0583 - acc: 0.6114 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 7/10
359s - loss: 0.0750 - acc: 0.6110 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 8/10
359s - loss: 0.0567 - acc: 0.6117 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 9/10
360s - loss: 0.0558 - acc: 0.6117 - val_loss: 0.0608 - val_acc: 0.6140
Epoch 10/10
359s - loss: 0.0564 - acc: 0.6117 - val_loss: 0.0608 - val_acc: 0.6140
Evaluate train
Evaluate validation
train model
Train on 8789 samples, validate on 4891 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[25600,512]
	 [[Node: mul_47 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](sub_30, gradients_1/MatMul_4_grad/MatMul_1)]]

Caused by op 'mul_47', defined at:
  File "/Users/kyledorman/anaconda/envs/carnd/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kyledorman/anaconda/envs/carnd/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kyledorman/anaconda/envs/carnd/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/kyledorman/anaconda/envs/carnd/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/kyledorman/anaconda/envs/carnd/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/kyledorman/anaconda/envs/carnd/lib/pyth

In [ ]:
from bc_helper.full_path import full_path
for k, model in {'orig': orig_results[0], 'smooth': smooth_results[0] }.items():
    model.save(full_path(k + "_model.h5"))

In [15]:
from bc_helper.load import _all_starter_data

df = _all_starter_data()
print('len(df):', len(df))
left = []
right = []
center = []
for index, row in df.iterrows():
    if index == 4:
        break
    left.append(pd.Series([row['steering'] - 0.5, row['left']]))
    right.append(pd.Series([row['steering'] + 0.5, row['right']]))
    center.append(pd.Series([row['steering'], row['center']]))
    
left = pd.DataFrame(left)
right = pd.DataFrame(right)
center = pd.DataFrame(center)
new = pd.concat([left, right, center])
new.columns = ['steering', 'center']
print(new)

len(df): 8036
   steering                                  center
0      -0.5    IMG/left_2016_12_01_13_30_48_287.jpg
1      -0.5    IMG/left_2016_12_01_13_30_48_404.jpg
2      -0.5    IMG/left_2016_12_01_13_31_12_937.jpg
3      -0.5    IMG/left_2016_12_01_13_31_13_037.jpg
0       0.5   IMG/right_2016_12_01_13_30_48_287.jpg
1       0.5   IMG/right_2016_12_01_13_30_48_404.jpg
2       0.5   IMG/right_2016_12_01_13_31_12_937.jpg
3       0.5   IMG/right_2016_12_01_13_31_13_037.jpg
0       0.0  IMG/center_2016_12_01_13_30_48_287.jpg
1       0.0  IMG/center_2016_12_01_13_30_48_404.jpg
2       0.0  IMG/center_2016_12_01_13_31_12_937.jpg
3       0.0  IMG/center_2016_12_01_13_31_13_037.jpg
